In [1]:
import sys
import io
import traceback
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler,LabelEncoder
from sklearn.decomposition import PCA
import sqlite3
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
import datetime as dt
from sklearn import linear_model
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
import json
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import openai
import pandas as pd
import sqlite3
from openai.api_resources.completion import Completion
import api_call

In [2]:
# Connect to the sqlite db file and retrieve data as Pandas data frame.
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
sql = "select * from fires"
# df = pd.read_sql_query(sql, cnx)

In [3]:
df = pd.read_sql_query(sql, cnx)
df.head()
str(df.columns)

"Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape'], dtype='object')"

In [4]:
df_interest = df[["FIRE_YEAR","DISCOVERY_DOY","DISCOVERY_TIME","CONT_DOY","CONT_TIME","STAT_CAUSE_CODE","STAT_CAUSE_DESCR","FIRE_SIZE","LATITUDE","LONGITUDE","STATE","COUNTY","FIPS_NAME","DISCOVERY_DATE"]]
df_interest.isna().sum()

FIRE_YEAR                0
DISCOVERY_DOY            0
DISCOVERY_TIME      882638
CONT_DOY            891531
CONT_TIME           972173
STAT_CAUSE_CODE          0
STAT_CAUSE_DESCR         0
FIRE_SIZE                0
LATITUDE                 0
LONGITUDE                0
STATE                    0
COUNTY              678148
FIPS_NAME           678148
DISCOVERY_DATE           0
dtype: int64

In [5]:
print(f"Number of removed duplicate rows: {df_interest.duplicated().sum()}")
df_interest = df_interest.drop_duplicates()

Number of removed duplicate rows: 3525


In [6]:
df_interest["combined_date_dis"] = df_interest["FIRE_YEAR"]*1000 + df_interest["DISCOVERY_DOY"]
df_interest["combined_date_dis"] = pd.to_datetime(df_interest["combined_date_dis"], format = "%Y%j")
df_interest = df_interest.set_index("combined_date_dis")
df_interest.head()

,FIRE_YEAR,DISCOVERY_DOY,DISCOVERY_TIME,CONT_DOY,CONT_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_NAME,DISCOVERY_DATE
combined_date_dis,,,,,,,,,,,,,,
2005-02-02,2005,33,1300,33.0,1730,9.0,Miscellaneous,0.10,40.036944,-121.005833,CA,63,Plumas,2453403.5
2004-05-12,2004,133,0845,133.0,1530,1.0,Lightning,0.25,38.933056,-120.404444,CA,61,Placer,2453137.5
2004-05-31,2004,152,1921,152.0,2024,5.0,Debris Burning,0.10,38.984167,-120.735556,CA,17,El Dorado,2453156.5
2004-06-28,2004,180,1600,185.0,1400,1.0,Lightning,0.10,38.559167,-119.913333,CA,3,Alpine,2453184.5
2004-06-28,2004,180,1600,185.0,1200,1.0,Lightning,0.10,38.559167,-119.933056,CA,3,Alpine,2453184.5


In [7]:
# df = df_interest

In [8]:
df.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape'], dtype='object')

In [9]:


# Set up OpenAI API key
openai.api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxx'



In [11]:
f = open("columns.txt", "r")
#summarise

In [12]:
col_txt = f.read()

In [13]:


def query_database(sql_query):
    """
    Executes a SQL query against the SQLite database.
    """
    return pd.read_sql_query(sql_query, cnx)

def generate_sql_query(messages):
    """
    Uses OpenAI's GPT-4 to generate a SQL query based on a natural language prompt.
    """
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
    )
    return response.choices[0].message['content'].strip()


def run_tool(json_query):
    """
    Uses OpenAI's GPT-4 to generate a SQL query based on a natural language prompt.
    """
    if(json_query["tool_name"] == "Query_Database"):
        try:
            df1 = query_database(json_query["command"])
            print(df1)
            data = df1.head().to_dict(orient='records')  # Converts the DataFrame head to a list of dictionaries
            return data
        except:
            print("Incorrect sql command. Trying again")
            return "Retry SQL Query"
    if(json_query["tool_name"] == "Generate_Final_Answer"):
        return "Finish"
        

In [14]:
import openai

def llm_generate_plan(query):
    """
    Generates a detailed plan using an LLM based on the provided query.
    
    Args:
        query (str): A natural language description of the task to be planned.
    
    Returns:
        list: A list of actionable steps derived from the LLM's response.
    """
    # Send the query to the LLM
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Specify the appropriate model
#             temperature = 0.7,
            messages=[
                {"role": "user", "content": query}
            ]

        )
        plan_text = response['choices'][0]['message']['content']
#         print("Full response from LLM:", plan_text)
        
        # Process the response into a structured plan
        # Here you'd parse the text into discrete steps; this example assumes the LLM returns well-structured text
        steps = plan_text.split('\n')  # Simple split by newline; adjust based on your actual LLM response structure
        
#         return steps
        return plan_text
    except Exception as e:
        print("Error in contacting the LLM:", e)
        return []



In [15]:
import openai
import yaml

def load_plugin_schema(plugin_name):
    """
    Loads the plugin schema from a YAML file.
    
    Args:
        plugin_name (str): The name of the plugin.
        
    Returns:
        dict: The plugin schema including name, arguments, and metadata.
    """
    with open(f"{plugin_name}.yaml", 'r') as file:
        plugin_schema = yaml.safe_load(file)
    return plugin_schema

def generate_code(plan_steps, plugin_schemas):
    """
    Generates Python code based on the given plan using specified plugins. Follow the plan as specified. Do not use additional models. Do as specified. Do not assume anything. If you need any variable, process it and used. Define variables concretely.
    
    Args:
        plan_steps (list): Steps defined by the planner.
        plugin_schemas (dict): Dictionary of loaded plugin schemas.
    
    Returns:
        str: Generated Python code.
    """
    code = ""
    code_prompt =  '''<s> [INST]
    Generate Python code for the given request. Import the necessary libraries first. Ensure that the generated code is ready for direct execution without requiring predefined variables outside of this context. Do not use the fbprophet package; use the Prophet package instead for trend analysis. Use from prophet import Prophet
    DATABASE DETAILS:
    - Database: SQLite
    - Table Name: fires stored in df
    - cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
    - USE cnx to query the database using sqlite commands


    MEMORY:
    Context: The 'fires' table contains a comprehensive spatial database of wildfires that occurred in the United States from 1992 to 2015. This data supports national Fire Program Analysis (FPA) systems. The records, sourced from federal, state, and local fire organizations, include essential elements such as discovery date, final fire size, and point locations precise to at least the Public Land Survey System (PLSS) section (1-square mile grid). The data conforms to standards set by the National Wildfire Coordinating Group (NWCG) and includes error-checking and redundancy removal. This data publication, known as the Fire Program Analysis fire-occurrence database (FPA FOD), comprises 1.88 million geo-referenced wildfire records, representing 140 million acres burned over a 24-year period.

    Column Information:

    '''
    code_prompt+= col_txt
    code_prompt+='''
    TOOLS
    - Generate_Final_Answer: Use if answer to User's question can be given with MEMORY and data we have. Use this after thorough data analysis
    - Query_Database: Write only one SQL Query to query the Database. This can only execute one query at a time
    - PCA_Analysis: Find variance of the principal components
    - Prophet_module: Implement prophet module to find the trends and trend change points. Use "from prophet import Prophet"
    - Remove_duplicates: Remove duplicate rows
    - convert datetime to required format
    - extract the necessary metrics and store in dataframe
    - Classifier - XGBClassifier - use variable name - xgb_model
    - Label Encoder - encode when needed and decode when predicting using inverse transform or decoder
    - RandomForestClassifier - use variable name - rf_model
    - ClassificationReport
    - Convert Julian time
        
    CODE GENERATION SPECIFICATIONS:
    1. Define all intermediate data structures or variables explicitly.
    2. Ensure all necessary libraries are imported at the beginning of the script.
    3. Write clear comments within the code to describe each operation.
    4. For each tool mentioned, include a code snippet that initializes the tool, performs the operation, and stores or prints the result.
    5. Make sure to handle all data preprocessing within the script, including the conversion of date formats, encoding of categorical variables, and preparation of the dataset for machine learning models.
    6. Avoid these errors - DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:DISCOVERY_DATE: object

    [/INST]
    Previous steps/CODE:
    '''
    for step in plan_steps:
        if 'plugin' in step:
            plugin_name = step.split(':')[1].strip()
            if plugin_name in plugin_schemas:
                schema = plugin_schemas[plugin_name]
                code += f"# Using plugin: {plugin_name}\n"
                code += f"{schema['function_name']}({', '.join(f'{arg}={value}' for arg, value in schema['arguments'].items())})\n"
        else:
            # Generate general Python code based on LLM suggestions
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content":code_prompt},
                    {"role": "user", "content": step}
                ]
                
            )
            generated_code = response['choices'][0]['message']['content']
            code += f"{generated_code}\n"
            code_prompt+=step
            code_prompt+=code
    return code

def extract_python_code(text):
    """
    Extract Python code blocks from the provided text string.

    Args:
        text (str): A string containing mixed content including Python code blocks.

    Returns:
        list of str: A list containing the extracted Python code blocks.
    """
    # Split the text by triple backticks to find potential code blocks
    parts = text.split('```python')
    code_blocks = []

    # Iterate over parts that come after `python` indicated by backticks
    for part in parts[1:]:  # skip the first split as it won't contain code
        # Find the first occurrence of triple backticks which marks the end of the code block
        end_idx = part.find('```')
        if end_idx != -1:
            # Extract the code block up to the ending backticks
            code_blocks.append(part[:end_idx].strip())

    return code_blocks
def run_code_with_custom_globals(generated_code, local_environment):
    """
    Executes the provided Python code using custom globals and captures the output.
    
    Args:
        generated_code (str): Python code to be executed.
        local_environment (dict): Custom global variables for the execution environment.
    
    Returns:
        str: Output from executing the code.
    """
    old_stdout = sys.stdout
    redirected_output = sys.stdout = io.StringIO()

    try:
        exec(generated_code, local_environment)
    except Exception as e:
        print(f"Error during execution: {e}", file=sys.stdout)
    finally:
        sys.stdout = old_stdout

    output = redirected_output.getvalue()
    redirected_output.close()
    return output
#    Generate Python code for the given request. Import the necessary libraries. Generate only the code so that I can directly execute. Do not use fbprophet only use prophet
#     The database in discussion is df (pandas dataframe).Do not assume the existence of variables. Create them if required and use them. Use proper variable names as defined, do not assume variable names.
#     Clearly define the target variable name based on previous steps.
#         df_model["combined_date_dis"] = df_model["FIRE_YEAR"]*1000 + df_model["DISCOVERY_DOY"]
#         df_model["combined_date_dis"] = pd.to_datetime(df_model["combined_date_dis"], format = "%Y%j")
#         df_model["combined_date_dis"] = df_model.combined_date_dis.astype(str) + " " + df_model.DISCOVERY_TIME.str[:2] + ":" + df_model.DISCOVERY_TIME.str[2:]
#         df_model["combined_date_dis"] = pd.to_datetime(df_model["combined_date_dis"])
#         df_model["combined_date_con"] = df_model["FIRE_YEAR"]*1000 + df_model["CONT_DOY"]
#         df_model["combined_date_con"] = pd.to_datetime(df_model["combined_date_con"], format = "%Y%j",errors="ignore")
#         df_model["combined_date_con"] = df_model.combined_date_con.astype(str) + " " + df_model.CONT_TIME.str[:2] + ":" + df_model.CONT_TIME.str[2:]
#         df_model["combined_date_con"] = pd.to_datetime(df_model["combined_date_con"],errors="coerce")

In [16]:
# Step 1: Plan generation
prompt= '''<s> [INST]
You are a planning module specifically designed for data analysis tasks involving SQLite databases. You will receive natural language queries and must develop a clear, step-by-step plan for data analysis, directing the code generator on how to execute each step effectively. The plan should outline necessary SQL queries, data preprocessing steps, analytical methods, and final reporting formats, specifying all variable names and any new DataFrames or structures needed.

You will interact with a code generator or tool user to extract insights from a SQLite database. Ensure each instruction is precise and clear to facilitate accurate code generation. Ensure each variable is also properly defined and formatted so that the code generator makes few mistakes. Make sure you use the column names existing in the dataframe based on previous planning steps.

TASK: Draft a comprehensive, step-by-step plan to perform data analysis on the specified SQLite table named 'fires'. This table contains extensive data on wildfires. The plan should outline the necessary SQL queries, data preprocessing steps, analytical methods, and final reporting formats. Mention all variable names clearly and specify any new DataFrames or structures needed. Do not assume functions and keep changing variable names.
Be very clear of any variable names you provide. Do not be vague with respect to planning. Use the column names provided in the prompt.
DATABASE DETAILS:
- Database: SQLite
- Table Name: fires

MEMORY:
Context: The 'fires' table contains a comprehensive spatial database of wildfires that occurred in the United States from 1992 to 2015. This data supports national Fire Program Analysis (FPA) systems. The records, sourced from federal, state, and local fire organizations, include essential elements such as discovery date, final fire size, and point locations precise to at least the Public Land Survey System (PLSS) section (1-square mile grid). The data conforms to standards set by the National Wildfire Coordinating Group (NWCG) and includes error-checking and redundancy removal. This data publication, known as the Fire Program Analysis fire-occurrence database (FPA FOD), comprises 1.88 million geo-referenced wildfire records, representing 140 million acres burned over a 24-year period.

Column Information:
'''
prompt+= col_txt
prompt+='''
TOOLS
- Print Final Answer: This is not a function. Treat this a plan termination. Just print the results. Use if answer to User's question can be given with MEMORY and data we have. Use this after thorough data analysis
- Query_Database: Write only one SQL Query to query the Database. This can only execute one query at a time
- PCA_Analysis: Find variance of the principal components
- Prophet_module: Implement prophet module to find the trends and trend change points. (from prophet import Prophet)  Convert the inputs into daily sum and change the column names to apply fpprophet.  Convert the inputs into daily sum and change the column names to apply fpprophet. Extract yearly trends.
- from prophet.plot import add_changepoints_to_plot #to find the trends in an automated fashion
- Remove_duplicates: Remove duplicate rows
- LinearRegression - LinearRegression()
- convert datetime to required format - from 223453464 to mmddyyyy
- extract the necessary metrics and store in dataframe
- Classifier - XGBClassifier (before mentioning the classifier, convert all the columns into the correct datatype to avoid errors like- Error during execution: could not convert string to float: '01/01/1970' )
- Label Encoder - encode when needed and decode when predicting using inverse transform or decoder
- RandomForestClassifier - use variable name - rf_model
- Classification_Report: Generate reports on classifier performance.
- Inverse transform
- Data_Encoding: Use Label Encoders for categorical data and handle inverse transformations.
- Split date format into week, month, day - for integer processing
- replace null type attributes with a suitable data type and value




Remember, your goal is to provide clear, executable instructions that a code generator can follow to generate code. Be very clear and define variables whenever needed. Use in-context example as a sample.

In-context examples:

User: 'Given the size, location and date, Predict the cause of a fire wildfire.'
Assistant:
1. Data Preparation:
Load and Clean Data: Import the dataset and select relevant columns such as 'FIRE_YEAR', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'DISCOVERY_DATE', 'CONT_DOY', 'CONT_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'FIRE_SIZE', 'LATITUDE', 'LONGITUDE', 'STATE', and 'FIPS_NAME'. Remove duplicate records to ensure data quality.
2. Feature Engineering:
Date and Time Conversion: Convert 'FIRE_YEAR' and 'DISCOVERY_DOY' to a datetime format to create a 'combined_date_dis' column. Similarly, combine 'FIRE_YEAR' and 'CONT_DOY' to create 'combined_date_con', ensuring both discovery and containment times are included.
Calculate Duration: Compute the duration of the fire by subtracting the discovery datetime from the containment datetime. Extract additional temporal features like weekday and month from the discovery date.
3. Exploratory Data Analysis (EDA):
Check Data Balance: Assess the balance of the categories within 'STAT_CAUSE_DESCR' and plan for potential data resampling if significant imbalances are detected.
Correlation Analysis: Calculate and visualize the correlation matrix for all numerical features to identify any strong correlations or redundancies. Drop highly correlated or redundant features to simplify the model.
4. Feature Engineering Continued:
Handle Categorical Data: Apply label encoding to categorical variables such as 'STATE' and 'FIPS_NAME' due to their high cardinality, which makes one-hot encoding impractical.
Feature Selection: Select significant features for the model, focusing on those most relevant for predicting fire causes, such as location, size, and date-derived features.
Instead of using STAT_CAUSE_CODE as label, implement LE for STAT_CAUSE_DESCR because STAT_CAUSE_CODE starts from 1.
Model Preparation:
5. Data Splitting: Divide the data into training and testing sets to ensure a robust evaluation of the model's predictive performance.
Handle Missing Data: Decide on strategies for handling missing data, whether imputing values or dropping rows, especially for key features used in the models.
Model Training and Evaluation:
6. Choose Algorithm: Train several machine learning model XGBoost, which are well-suited for handling the categorical nature and complexity of the dataset.
7. Feature Importance: Analyze the importance of different features in the model to gain insights into what factors most influence the prediction of fire causes.
Model Insights: Provide comprehensive insights and recommendations based on the model outcomes, highlighting key predictive factors and potential areas for further research or operational integration.


User: Have wildfires become more or less frequent over time?
Assistant:
1. Data Preparation
Load the dataset into a DataFrame named df_interest by directly querying Select relevant columns related to fire occurrences and timing, such as 'FIRE_YEAR', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'CONT_DOY', 'CONT_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'FIRE_SIZE', 'LATITUDE', 'LONGITUDE', 'STATE', 'COUNTY', 'FIPS_NAME', and 'DISCOVERY_DATE'. Store these in df_interest.
Check for and summarize any missing values in df_interest using the isna().sum() method.
Remove duplicate rows to clean the data, noting how many duplicates were removed with a print statement.
2. Date Conversion
Create a new column 'combined_date_dis' in df_interest that combines these columns into a full date format using pandas' to_datetime function, considering 'FIRE_YEAR' as the year and 'DISCOVERY_DOY' as the day of the year.
3. Data Aggregation
Set the new datetime column as the index of df_interest.
Aggregate data to analyze fire counts on a monthly and yearly basis using groupby and count methods. Store these aggregates in monthly_trend and yearly_trend, respectively, and then plot these trends.
4. Trend Analysis Using Prophet
Prepare the data for Prophet by resetting the index and renaming columns to 'ds' (date) and 'y' (metric, e.g., count of fires). This is necessary for compatibility with the Prophet package.
Define a function prophet_model that sets up and runs a Prophet model. The function should accept parameters like growth assumptions, seasonality, and the number of changepoints, and output both the model and its forecasts.
Apply the prophet_model function to the daily aggregated fire count data.
5. Change Point Analysis
Use Prophet's output to visualize components and trend change points. Plot the overall forecast and highlight changepoints.
6. Linear Regression to find change point
Divide the dataset into portions lower and upper of the change point using trend = forecast[["ds","trend"]] and using changepoints.to_list() module.
Analyze trends before and after the first changepoint using linear regression to determine changes in the slope of the trend lines. Convert datetime to ordinal using the required libraries.
6. Output Results
Calculate and print the slopes of the trend lines before and after the changepoint to discuss whether wildfires have become more or less frequent over time.

User: What counties are the most and least fire-prone? \n
Assistant: 
1. Execute an SQL query to extract records from the fire incidents database. Retrieve all relevant columns for state, county, and monthly fire counts for each county. Store the results in a DataFrame named monthly_fire_counts.
2. In the monthly_fire_counts DataFrame, merge the state and county names into a single column named 'State_County'. This facilitates easier aggregation and analysis.
3. Using SQL, aggregate the total number of fires, the cumulative fire duration, and the total area affected by fires for each county. Store these aggregated metrics in a new DataFrame named county_fire_stats.
4. Standardize the metrics (total fires, duration, and area) in the county_fire_stats DataFrame using z-scores to allow comparison across different scales and distributions.
5. Apply PCA to the standardized metrics in county_fire_stats to reduce dimensionality and identify the principal components that explain the most variance in the data.
6. Analyze the explained variance of each principal component to identify the most significant component that correlates with fire proneness.
7. Determine the counties with the highest and lowest scores on the principal component that shows the highest variance. These scores will indicate the most and least fire-prone counties.
8. Print the names of the counties that are identified as the most and least fire-prone based on the analysis.


[/INST]

User:
'''
# query = 'What counties are the most and least fire-prone?'
# query = 'Have wildfires become more or less frequent over time?'
# query = 'Given the size, location and date, Predict the cause of a fire wildfire.'
# query = 'What is this dataset about?'
# query = 'What questions can I ask?'
# query = 'What year had the highest number of wildfires?'
# query = 'Estimate the total property damage caused by wildfires in 2015.'
query = 'What factors are most predictive of fire containment times?' #not working
# query = 'Based on historical data, predict the likelihood of a major wildfire in Texas in the next year.'
prompt+= query
plan_txt = llm_generate_plan(prompt)
print(plan_txt)
# print("Generated Plan Steps:")
#TASK: Draft a comprehensive, step-by-step plan to perform data analysis on a specified table. Mention the variable names clearly and the new dataframes needed for the code generator to understand.


# Assistant:
# 1. Extract monthly counts of fires for each county using SQL:
#    - `monthly_fires_df = QUERY('SELECT state, county, COUNT(*) AS monthly_fire_count, strftime('%m', discovery_date) AS month FROM fires GROUP BY state, county, month');`
# 2. Combine state and county into a unique identifier:
#    - `monthly_fires_df['county_identifier'] = monthly_fires_df['state'] + ', ' + monthly_fires_df['county'];`
# 3. Retrieve and rank the top 10 counties by total fire duration, size, and counts:
#    - `top_duration_df = QUERY('SELECT county_identifier, SUM(duration) AS total_duration FROM monthly_fires_df GROUP BY county_identifier ORDER BY total_duration DESC LIMIT 10');`
#    - Similar queries for fire size and counts.
# 4. Combine these metrics into a single DataFrame and standardize the values:
#    - `combined_metrics_df = MERGE(top_duration_df, top_size_df, top_counts_df);`
#    - `standardized_metrics_df = STANDARDIZE(combined_metrics_df);`
# 5. Implement PCA on the standardized data and identify the principal component with the highest variance.
#    - `pca_results = PCA_ANALYSIS(standardized_metrics_df);`
# 6. Output the counties from the component showing the highest variance as the most and least fire-prone.
#    - `final_result = IDENTIFY_EXTREMES(pca_results);`
#    - `PRINT('Most and Least Fire-Prone Counties: ', final_result);`
# Assistant: Extract monthly count of fires for each county. \n 
# Combine State and Counties \n
# Retrieve the top 10 counties in terms of total fire duration. \n
# Retrieve the top 10 counties in terms of total fire size. \n
# Retrieve the top 10 counties in terms of fire counts. \n
# Combine the metrics total fire duration, total fire size,fire counts into the same data frame and standardize the values \n
# Implement PCA and calculate the variances of the principal components. \n
# Retrieve the component that has the highest varience. \n
# Print the counties that are the least and the most fire-prone. \n

'''
1. Extract monthly count of fires for each county. \n 
2. Combine State and Counties \n
3. Retrieve the top 10 counties in terms of total fire duration. \n
4. Retrieve the top 10 counties in terms of total fire size. \n
5. Retrieve the top 10 counties in terms of fire counts. \n
6. Combine the metrics total fire duration, total fire size,fire counts into the same data frame and standardize the values \n
7. Implement PCA and calculate the variances of the principal components. \n
8. Retrieve the component that has the highest variance. \n
9. Print the counties that are the least and the most fire-prone. \n

'''



Assistant:
1. Data Preparation:
Load the dataset from the 'fires' table into a DataFrame, including columns 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'STAT_CAUSE_DESCR', 'LATITUDE', 'LONGITUDE', 'STATE', and 'FIPS_NAME'. 
Remove any duplicate records to ensure data quality.
2. Feature Engineering:
Convert 'DISCOVERY_DATE' and 'CONT_DATE' columns into datetime format for better temporal analysis and feature extraction.
Calculate the duration of each fire by subtracting the discovery date from the containment date to create a new column 'FIRE_DURATION'.
Extract additional temporal features like day of the week and month from the 'DISCOVERY_DATE' column to enrich the dataset.
3. Exploratory Data Analysis (EDA):
Check for missing values in the dataset and decide on appropriate handling strategies such as imputation or removal.
Perform correlation analysis among numerical features like 'FIRE_SIZE', 'LATITUDE', and 'LONGITUDE' to identify rela

'\n1. Extract monthly count of fires for each county. \n \n2. Combine State and Counties \n\n3. Retrieve the top 10 counties in terms of total fire duration. \n\n4. Retrieve the top 10 counties in terms of total fire size. \n\n5. Retrieve the top 10 counties in terms of fire counts. \n\n6. Combine the metrics total fire duration, total fire size,fire counts into the same data frame and standardize the values \n\n7. Implement PCA and calculate the variances of the principal components. \n\n8. Retrieve the component that has the highest variance. \n\n9. Print the counties that are the least and the most fire-prone. \n\n\n'

In [17]:
import re

def extract_numbered_list_items(text):
    """
    Extracts numbered list items from the provided text.

    Args:
        text (str): The input text from which numbered list items will be extracted.
    
    Returns:
        list: A list of strings, each a numbered list item containing numbers.
    """
    # Regex to find lines starting with one or more digits followed by a period and optional whitespace
    pattern = r'\n?(\d+\.\s+.*?(?=\n\d+\.|\n\n|\Z))'
    numbered_items = re.findall(pattern, text, flags=re.DOTALL)

    return [item.strip() for item in numbered_items]
plan_steps = extract_numbered_list_items(plan_txt)

In [18]:
plan_steps

["1. Data Preparation:\nLoad the dataset from the 'fires' table into a DataFrame, including columns 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'STAT_CAUSE_DESCR', 'LATITUDE', 'LONGITUDE', 'STATE', and 'FIPS_NAME'. \nRemove any duplicate records to ensure data quality.",
 "2. Feature Engineering:\nConvert 'DISCOVERY_DATE' and 'CONT_DATE' columns into datetime format for better temporal analysis and feature extraction.\nCalculate the duration of each fire by subtracting the discovery date from the containment date to create a new column 'FIRE_DURATION'.\nExtract additional temporal features like day of the week and month from the 'DISCOVERY_DATE' column to enrich the dataset.",
 "3. Exploratory Data Analysis (EDA):\nCheck for missing values in the dataset and decide on appropriate handling strategies such as imputation or removal.\nPerform correlation analysis among numerical features like 'FIRE_SIZE', 'LATITUDE', and 'LONGITUDE' to identif

In [19]:
import openai
import yaml
local_environment = {'df': df_interest}

def load_plugin_schema(plugin_name):
    """
    Loads the plugin schema from a YAML file.
    
    Args:
        plugin_name (str): The name of the plugin.
        
    Returns:
        dict: The plugin schema including name, arguments, and metadata.
    """
    with open(f"{plugin_name}.yaml", 'r') as file:
        plugin_schema = yaml.safe_load(file)
    return plugin_schema

def extract_python_code(text):
    """
    Extract Python code blocks from the provided text string.

    Args:
        text (str): A string containing mixed content including Python code blocks.

    Returns:
        list of str: A list containing the extracted Python code blocks.
    """
    # Split the text by triple backticks to find potential code blocks
    parts = text.split('```python')
    code_blocks = []

    # Iterate over parts that come after `python` indicated by backticks
    for part in parts[1:]:  # skip the first split as it won't contain code
        # Find the first occurrence of triple backticks which marks the end of the code block
        end_idx = part.find('```')
        if end_idx != -1:
            # Extract the code block up to the ending backticks
            code_blocks.append(part[:end_idx].strip())

    return code_blocks

In [35]:
def regenerate_code_with_reflection(original_code, error_message,code_prompt):
    """
    Use LLM to regenerate the code based on the error encountered.
    """
    prompt = f"Correct this Python code which caused an error:\nError: {error_message}\nCode: {original_code}\n# Use {code_prompt} for reference to correct it"
    
    prompt = (
        f"### Error Correction Task\n"
        f"## Context:\n"
        f"- An error occurred during the execution of this Python code, which is part of a larger project involving data analysis.\n"
        f"- Below is the erroneous code along with the error message and the code context from the broader project.\n"
        f"- Your task is to correct the code to resolve the error and ensure it aligns with the project's objectives and coding standards.\n\n"
        f"## Error Details:\n"
        f"- Error Message: {error_message}\n\n"
        f"## Original Code Causing the Error:\n"
        f"```python\n"
        f"{original_code}\n"
        f"```\n\n"
        f"## Project Code Context (Reference):\n"
        f"```python\n"
        f"{code_prompt}\n"
        f"```\n\n"
        f"## Instructions:\n"
        f"- Correct the original code to eliminate the error.\n"
        f"- Ensure the corrected code integrates seamlessly with the provided project code context.\n"
        f"- Maintain consistency with the coding practices and standards used in the project.\n"
        f"- Provide explanations for the changes made to help understand the correction process.\n"
        f"- Make sure the type matches correctly for the classifiers"
        f"- Import the correct/required libraries"
        f"- Where possible, convert data types explicitly to compatible types before operation. For example, convert timedelta to numeric types (such as total seconds) if the operation requires numeric computation."
        f"- For variables like date, create new columns to support multiple datatypes like datetimelike, int, and so on"
        f"- For variable type errors, try to clean the data before predictive modelling"
        f"- Check the existence of the columns before dropping them"
        f"- Specify datetime formats clearly - for example format='%m%d%Y for better processing"
        f"- For column type errors, recheck previous code to properly correct new code"
        f"<INST> Generate only python code <\INST>"
    )

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":prompt},
        ]
    )
    corrected_code = response['choices'][0]['message']['content']
#     print("Corrected_code")
#     print(corrected_code)
    return corrected_code
def run_code_with_custom_globals(generated_code):
    """
    Executes the provided Python code using custom globals and captures the output.
    
    Args:
        generated_code (str): Python code to be executed.
        local_environment (dict): Custom global variables for the execution environment.
    
    Returns:
        str: Output from executing the code.
    """
    old_stdout = sys.stdout
    redirected_output = sys.stdout = io.StringIO()   
    tmp = ""


    try:
        exec(generated_code, local_environment)
    except Exception as e:
        print(f"Error during execution: {e}", file=sys.stdout)
        
    finally:
        sys.stdout = old_stdout

    output = redirected_output.getvalue()
    redirected_output.close()
    return output

In [36]:

def generate_run_code(plan_steps, plugin_schemas):
    print("entered")
    print(plan_steps)
    """
    Generates Python code based on the given plan using specified plugins. Follow the plan as specified. Do not use additional models. Do as specified. Do not assume anything. If you need any variable, process it and used. Define variables concretely.
    
    Args:
        plan_steps (list): Steps defined by the planner.
        plugin_schemas (dict): Dictionary of loaded plugin schemas.
    
    Returns:
        str: Generated Python code.
    """
    code = ""
    code_prompt =  '''<s> [INST]
    Generate Python code for the given request. Import the necessary libraries first. Ensure that the generated code is ready for direct execution without requiring predefined variables outside of this context. Do not use the fbprophet package; use the Prophet package instead for trend analysis. Use from prophet import Prophet
    DATABASE DETAILS:
    - Database: SQLite
    - Table Name: fires stored in df
    - cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
    - USE cnx to query the database using sqlite commands


    MEMORY:
    Context: The 'fires' table contains a comprehensive spatial database of wildfires that occurred in the United States from 1992 to 2015. This data supports national Fire Program Analysis (FPA) systems. The records, sourced from federal, state, and local fire organizations, include essential elements such as discovery date, final fire size, and point locations precise to at least the Public Land Survey System (PLSS) section (1-square mile grid). The data conforms to standards set by the National Wildfire Coordinating Group (NWCG) and includes error-checking and redundancy removal. This data publication, known as the Fire Program Analysis fire-occurrence database (FPA FOD), comprises 1.88 million geo-referenced wildfire records, representing 140 million acres burned over a 24-year period.

    Column Information:

    '''
    code_prompt+= col_txt
    code_prompt+='''
    TOOLS
    - Print Final Answer: This is not a function. Treat this a plan termination. Just print the results. Use if answer to User's question can be given with MEMORY and data we have. Use this after thorough data analysis
    - Query_Database: Write only one SQL Query to query the Database. This can only execute one query at a time
    - PCA_Analysis: Find variance of the principal components
    - Prophet_module: Implement prophet module to find the trends and trend change points. (from prophet import Prophet)  Convert the inputs into daily sum and change the column names to apply fpprophet.  Convert the inputs into daily sum and change the column names to apply fpprophet. Extract yearly trends. Use plot_components if needed.
    - Remove_duplicates: Remove duplicate rows
    - Convert Julian date format into Gregorian date format.
        df_interest["combined_date_dis"] = df_interest["FIRE_YEAR"]*1000 + df_interest["DISCOVERY_DOY"]
        df_interest["combined_date_dis"] = pd.to_datetime(df_interest["combined_date_dis"], format = "%Y%j")
        df_interest = df_interest.set_index("combined_date_dis")
        df_interest.head()
    - extract the necessary metrics and store in dataframe
    - Classifier - XGBClassifier (before mentioning the classifier, convert all the columns into the correct datatype to avoid errors like- Error during execution: could not convert string to float: '01/01/1970' )
    - Label Encoder - encode when needed and decode when predicting using inverse transform or decoder
    - RandomForestClassifier - use variable name - rf_model
    - Classification_Report: Generate reports on classifier performance.
    - Inverse transform
    - Data_Encoding: Use Label Encoders for categorical data and handle inverse transformations.
    - Split date format into week, month, day - for integer processing
    - Remove None by dropna
    - Pandas - import pandas as pd
    - Analysis of trend change points 
        trend = forecast[["ds","trend"]]
        lower_portion = trend[(trend["ds"]<m.changepoints.to_list()[0])]
        upper_portion = trend[(trend["ds"]>m.changepoints.to_list()[0])]

        # Calculate slopes with using linear Regression.
        reg_lower = linear_model.LinearRegression()
        reg_upper = linear_model.LinearRegression()

        reg_lower.fit(pd.to_datetime(lower_portion['ds']).map(dt.datetime.toordinal).values.reshape(-1, 1), lower_portion['trend'].values)
        reg_upper.fit(pd.to_datetime(upper_portion['ds']).map(dt.datetime.toordinal).values.reshape(-1, 1), upper_portion['trend'].values)

        print(f"Slope before changepoint({m.changepoints.to_list()[0]}): {reg_lower.coef_}")
        print(f"Slope before changepoint({m.changepoints.to_list()[0]}): {reg_upper.coef_}")
        
    CODE GENERATION SPECIFICATIONS:
    1. Define all intermediate data structures or variables explicitly.
    2. Ensure all necessary libraries are imported at the beginning of the script.
    3. Write clear comments within the code to describe each operation.
    4. For each tool mentioned, include a code snippet that initializes the tool, performs the operation, and stores or prints the result.
    5. Make sure to handle all data preprocessing within the script, including the conversion of date formats, encoding of categorical variables, and preparation of the dataset for machine learning models.
    6. Avoid these errors - DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:DISCOVERY_DATE: object
    7. Use Previous steps/CODE to define the variable names properly and reuse them if needed
    8. Avoid variable name errors by carefully using <CODE> to define the variables
    9. Extract relevant features which will be useful for prediction and they all follow the necessary range of values consistently
    10. replace null type attributes with a suitable data type and value
    11. Check within the prompt for the necessary column names. Do not assume column names

    [/INST]
    Previous steps <CODE>:
    '''
    counter = 0
    f = open("execution_results.txt", "w")
    f_error = open("error_results.txt","w")
    f_code = open("code_txt.txt","w")
    error_counter =0
    print(counter)
#     local_environment = {'df': df}
    for step in plan_steps:
        counter+=1
        print(counter)
        if 'plugin' in step:
            plugin_name = step.split(':')[1].strip()
            if plugin_name in plugin_schemas:
                schema = plugin_schemas[plugin_name]
                code += f"# Using plugin: {plugin_name}\n"
                code += f"{schema['function_name']}({', '.join(f'{arg}={value}' for arg, value in schema['arguments'].items())})\n"
        else:
            # Generate general Python code based on LLM suggestions
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content":code_prompt},
                    {"role": "user", "content": step}
                ]
            )
            generated_code = response['choices'][0]['message']['content']
            code += f"{generated_code}\n"
            code_formatted = f"{generated_code}\n"
            tmp = extract_python_code(code_formatted)
            
            if(len(tmp)==0):
                print("jalend")
                print(step)
                print(generated_code)
                break
            f_code.write(str(tmp[0])+"\n")    
#             print("Step asked")
#             print(step)
#             print("First generated code")
#             print(generated_code)
#             print("Code extracted first")
#             print(tmp)
#             execution_result = run_code_with_custom_globals(tmp[0])
            try:
                execution_result = run_code_with_custom_globals(tmp[0])
            except:
                print("error in tmp")
                print(tmp)
                break
#             print(execution_result)
            if('Error' in execution_result):
                print("Error detected")
                print(execution_result)
                f_error.write(str(execution_result) + "\n")
                error_counter+=1
                print(error_counter)
#                 print(execution_result)
                regenerated_code = regenerate_code_with_reflection(tmp[0],execution_result,code_prompt)
                code += f"{regenerated_code}\n"
                code_formatted = f"{regenerated_code}\n"
                tmp = extract_python_code(code_formatted)
                
#                 print("Code extracted after error")
#                 print(tmp)
                execution_result = run_code_with_custom_globals(tmp[0])
                try:
                    execution_result = run_code_with_custom_globals(tmp[0])
                    f_code.write(str(tmp[0])+"\n")
                    if('Error' in execution_result):
                        print("error still exists")
                        print(execution_result)
                        print(tmp[0])
                        break
                except:
                    print("error in  tmp error")
                    print(tmp)
                    print(regenerated_code)
                    print(code_formatted)
        

            code_prompt+=step
            code_prompt+=code_formatted
            f.write(str(execution_result) + "\n")
            #print(execution_result)
    f.close()
    f_error.close()
    f_code.close()
    return code



#    Generate Python code for the given request. Import the necessary libraries. Generate only the code so that I can directly execute. Do not use fbprophet only use prophet
#     The database in discussion is df (pandas dataframe).Do not assume the existence of variables. Create them if required and use them. Use proper variable names as defined, do not assume variable names.
#     Clearly define the target variable name based on previous steps.
#    10. After identifying and extracting the necessary features for modeling, systematically remove rows containing None values to ensure dataset integrity



plugin_schemas = {}
generated_code = generate_run_code(plan_steps, plugin_schemas)
# tmp = extract_python_code(generated_code)


entered
["1. Data Preparation:\nLoad the dataset from the 'fires' table into a DataFrame, including columns 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'STAT_CAUSE_DESCR', 'LATITUDE', 'LONGITUDE', 'STATE', and 'FIPS_NAME'. \nRemove any duplicate records to ensure data quality.", "2. Feature Engineering:\nConvert 'DISCOVERY_DATE' and 'CONT_DATE' columns into datetime format for better temporal analysis and feature extraction.\nCalculate the duration of each fire by subtracting the discovery date from the containment date to create a new column 'FIRE_DURATION'.\nExtract additional temporal features like day of the week and month from the 'DISCOVERY_DATE' column to enrich the dataset.", "3. Exploratory Data Analysis (EDA):\nCheck for missing values in the dataset and decide on appropriate handling strategies such as imputation or removal.\nPerform correlation analysis among numerical features like 'FIRE_SIZE', 'LATITUDE', and 'LONGITUDE' to i